In [2]:

import os
import numpy as np
import uuid
from pathlib import Path
from typing import List, Dict, Any
from dotenv import load_dotenv

# LangChain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embeddings and Vector Store
from sentence_transformers import SentenceTransformer
import chromadb

# LLM
from groq import Groq
import time

start_time = time.time()

# Load environment variables
load_dotenv()

print("✅ All imports successful")



def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✅ Loaded {len(documents)} pages")
        
        except Exception as e:
            print(f"  ❌ Error: {e}")
    
    print(f"\n✅ Total documents loaded: {len(all_documents)}")
    return all_documents

# ============================================================================
# 2. TEXT SPLITTING
# ============================================================================

def split_documents(documents, chunk_size=800, chunk_overlap=100):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]  # ✅ Fixed typo
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"✅ Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

# ============================================================================
# 3. EMBEDDING MANAGER
# ============================================================================

class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: Hugging Face model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"✅ Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"❌ Error loading model {self.model_name}: {e}")
            raise
    
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
        Returns:
            Numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f'Generating embeddings for {len(texts)} texts...')
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f'✅ Generated embeddings with shape: {embeddings.shape}')
        return embeddings

# ============================================================================
# 4. VECTOR STORE
# ============================================================================

class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = 'pdf_documents', 
                 persist_directory: str = "data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()
    
    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f'✅ Vector store initialized. Collection: {self.collection_name}')
            print(f'   Existing documents in collection: {self.collection.count()}')
        
        except Exception as e:
            print(f"❌ Error initializing vector store: {e}")
            raise
    
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"✅ Successfully added {len(documents)} documents to vector store")
            print(f"   Total documents in collection: {self.collection.count()}")
        
        except Exception as e:
            print(f"❌ Error adding documents to vector store: {e}")
            raise

# ============================================================================
# 5. RAG RETRIEVER
# ============================================================================

class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
    
    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
        
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"🔍 Retrieving documents for query: '{query[:50]}...'")
        print(f"   Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"✅ Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("⚠️ No documents found")
            
            return retrieved_docs
        
        except Exception as e:
            print(f"❌ Error during retrieval: {e}")
            return []

# ============================================================================
# 6. GROQ LLM CLIENT
# ============================================================================

class GroqClient:
    """Client to interact with Groq LLM API"""
    
    def __init__(self, api_key: str = None):
        """
        Initialize the Groq client
        
        Args:
            api_key: Groq API key (if None, reads from GROQ_API_KEY env variable)
        """
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        if not self.api_key:
            raise ValueError(
                "Groq API key not found. Set GROQ_API_KEY environment variable or pass api_key parameter."
            )
        
        self.client = self._initialize_client()
    
    def _initialize_client(self):
        """Initialize Groq client with API key"""
        try:
            print("Initializing Groq client...")
            client = Groq(api_key=self.api_key)
            print("✅ Groq client initialized successfully")
            return client
        except Exception as e:
            print(f"❌ Error initializing Groq client: {e}")
            raise
    
    def generate_response(
        self,
        query: str,
        contexts: List[Dict[str, Any]],
        model: str = "llama-3.3-70b-versatile",
        temperature: float = 0.3,
        max_tokens: int = 1000
    ) -> Dict[str, Any]:
        """
        Generate response using retrieved contexts
        
        Args:
            query: User's question
            contexts: List of retrieved documents from RAGRetriever
            model: Groq model to use
            temperature: Response creativity (0.0-1.0)
            max_tokens: Maximum response length
        
        Returns:
            Dictionary with answer and metadata
        """
        if not contexts:
            return {
                "answer": "I couldn't find any relevant information in the documents to answer your question.",
                "sources": [],
                "model": model,
                "num_contexts_used": 0
            }
        
        # Format contexts with numbering for citations
        formatted_contexts = []
        for i, ctx in enumerate(contexts, 1):
            source = ctx['metadata'].get('source_file', 'Unknown')
            page = ctx['metadata'].get('page', 'N/A')
            similarity = ctx.get('similarity_score', 0.0)
            
            formatted_contexts.append(
                f"[{i}] (Source: {source}, Page: {page}, Relevance: {similarity:.1%})\n{ctx['content']}"
            )
        
        context_text = "\n\n".join(formatted_contexts)
        
        # Create structured prompt
        prompt = f"""You are a helpful AI assistant. Answer the user's question based ONLY on the provided context from PDF documents.

IMPORTANT INSTRUCTIONS:
1. Only use information from the context below
2. Cite sources using [1], [2], [3] format after relevant statements
3. If the context doesn't contain enough information, clearly state that
4. Be concise but comprehensive
5. If you're uncertain, express appropriate confidence levels

CONTEXT:
{context_text}

QUESTION: {query}

ANSWER (with citations):"""
        
        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=max_tokens
            )
            
            answer = response.choices[0].message.content
            
            # Extract source information
            sources = [
                {
                    "id": i + 1,
                    "file": ctx['metadata'].get('source_file', 'Unknown'),
                    "page": ctx['metadata'].get('page', 'N/A'),
                    "similarity": ctx.get('similarity_score', 0.0)
                }
                for i, ctx in enumerate(contexts)
            ]
            
            return {
                "answer": answer,
                "sources": sources,
                "model": model,
                "num_contexts_used": len(contexts)
            }
        
        except Exception as e:
            print(f"❌ Error generating response: {e}")
            return {
                "answer": f"Error generating response: {str(e)}",
                "sources": [],
                "model": model,
                "num_contexts_used": 0
            }

# ============================================================================
# 7. RAG SYSTEM (ORCHESTRATOR)
# ============================================================================

class RAGSystem:
    """Complete RAG system that orchestrates all components"""
    
    def __init__(self, vector_store, embedding_manager, llm_client, retriever=None):
        """
        Initialize RAG system
        
        Args:
            vector_store: Vector store instance
            embedding_manager: Embedding manager instance
            llm_client: LLM client instance
            retriever: Optional pre-initialized retriever
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
        self.llm_client = llm_client
        self.retriever = retriever if retriever is not None else RAGRetriever(
            self.vector_store, self.embedding_manager
        )
        self.conversation_history = []
        
        print("✅ RAG System initialized successfully")
    
    def query(self, question: str, top_k: int = 5, score_threshold: float = 0.5, **llm_kwargs):
        """
        Query the RAG system
        
        Args:
            question: User's question
            top_k: Number of documents to retrieve
            score_threshold: Minimum similarity threshold
            **llm_kwargs: Additional arguments for LLM (model, temperature, etc.)
        
        Returns:
            Response dictionary with answer, sources, and metadata
        """
        print(f"\n{'─'*70}")
        print(f"📝 Query: {question}")
        print(f"{'─'*70}\n")
        
        # Retrieve contexts
        contexts = self.retriever.retrieve(
            question,
            top_k=top_k,
            score_threshold=score_threshold
        )
        
        if not contexts:
            print("⚠️ No relevant contexts found")
            response = {
                'answer': 'I could not find relevant information to answer this question.',
                'sources': [],
                'num_contexts_used': 0,
                'question': question
            }
        else:
            # Generate response
            response = self.llm_client.generate_response(
                query=question,
                contexts=contexts,
                **llm_kwargs
            )
            response['question'] = question
            response['retrieved_contexts'] = contexts
        
        # Track conversation
        self.conversation_history.append({
            'question': question,
            'answer': response.get('answer'),
            'sources': response.get('sources', []),
            'num_contexts': response.get('num_contexts_used', 0)
        })
        
        return response
    
    def display_response(self, response: dict):
        """Display response in a formatted way"""
        print("\n" + "="*70)
        print("✅ ANSWER")
        print("="*70)
        print(response.get('answer', 'No answer returned'))
        
        print("\n" + "="*70)
        print(f"📄 SOURCES ({response.get('num_contexts_used', 0)} documents)")
        print("="*70)
        
        sources = response.get('sources', [])
        if not sources:
            print("  No sources available")
        else:
            for src in sources:
                if isinstance(src, dict):
                    print(f"  [{src.get('id', '?')}] {src.get('file', 'Unknown')} "
                          f"(Page {src.get('page', 'N/A')}) - "
                          f"Relevance: {src.get('similarity', 0.0):.1%}")
                else:
                    print(f"  - {src}")
        
        print(f"\n🤖 Model: {response.get('model', 'N/A')} | "
              f"Contexts: {response.get('num_contexts_used', 0)}")
        print("="*70 + "\n")
    
    def get_stats(self):
        """Get session statistics"""
        if not self.conversation_history:
            return {
                "total_queries": 0,
                "total_contexts_retrieved": 0,
                "avg_contexts_per_query": 0.0
            }
        
        total_contexts = sum(c['num_contexts'] for c in self.conversation_history)
        return {
            "total_queries": len(self.conversation_history),
            "total_contexts_retrieved": total_contexts,
            "avg_contexts_per_query": total_contexts / len(self.conversation_history)
        }
    
    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []
        print("✅ Conversation history cleared")


# ============================================================================
# 8. MAIN EXECUTION
# ============================================================================

print("\n" + "="*70)
print("🚀 INITIALIZING RAG SYSTEM")
print("="*70 + "\n")

# Step 1: Load PDFs
print("Step 1: Loading PDFs...")
all_pdf_documents = process_all_pdfs("data/pdf")

# Step 2: Split into chunks
print("\nStep 2: Splitting documents...")
chunks = split_documents(all_pdf_documents, chunk_size=800, chunk_overlap=100)

# Step 3: Initialize Embedding Manager
print("\nStep 3: Initializing Embedding Manager...")
embedding_manager = EmbeddingManager()

# Step 4: Generate embeddings
print("\nStep 4: Generating embeddings...")
texts = [chunk.page_content for chunk in chunks]  # ✅ Fixed: Extract text from chunks
embeddings = embedding_manager.generate_embeddings(texts)

# Step 5: Initialize Vector Store
print("\nStep 5: Initializing Vector Store...")
vectorstore = VectorStore()

# Step 6: Add documents to vector store
print("\nStep 6: Adding documents to vector store...")
vectorstore.add_documents(chunks, embeddings)

# Step 7: Initialize Groq Client
print("\nStep 7: Initializing Groq LLM Client...")
groq_client = GroqClient()  # Reads GROQ_API_KEY from environment

# Step 8: Create RAG System
print("\nStep 8: Creating RAG System...")
rag_system = RAGSystem(
    vector_store=vectorstore,
    embedding_manager=embedding_manager,
    llm_client=groq_client
)

print("\n" + "="*70)
print("✅ SETUP COMPLETE - Ready to Query!")
print("="*70 + "\n")
elapsed_time = time.time() - start_time
print(f"\n⏱️ Total setup time: {elapsed_time:.2f} seconds")

ConfigError: unable to infer type for attribute "chroma_server_nofile"

In [6]:
# ============================================================================
# 9. EXAMPLE QUERIES
# ============================================================================

print("\n" + "="*70)
print("🤖 RUNNING TEST QUERIES")
print("="*70 + "\n")

test_queries = [
    "What are the main topics of artificial intelligence covered in the documents?",
    "Explain the key findings from the research papers about neural networks.",
    "What methodologies are discussed regarding agentic ai?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'━'*70}")
    print(f"QUERY {i}/{len(test_queries)}")
    print(f"{'━'*70}")
    
    response = rag_system.query(
        question=query,
        top_k=3,
        score_threshold=0.5,
        model="llama-3.3-70b-versatile",
        temperature=0.3,
        max_tokens=500
    )
    
    rag_system.display_response(response)

# ============================================================================
# 10. SESSION STATISTICS
# ============================================================================

print("\n" + "="*70)
print("📊 SESSION STATISTICS")
print("="*70)
stats = rag_system.get_stats()
print(f"Total Queries: {stats['total_queries']}")
print(f"Total Contexts Retrieved: {stats['total_contexts_retrieved']}")
print(f"Avg Contexts per Query: {stats['avg_contexts_per_query']:.1f}")
print("="*70 + "\n")

print("✅ RAG System ready for interactive queries!")


🤖 RUNNING TEST QUERIES


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUERY 1/3
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

──────────────────────────────────────────────────────────────────────
📝 Query: What are the main topics of artificial intelligence covered in the documents?
──────────────────────────────────────────────────────────────────────

🔍 Retrieving documents for query: 'What are the main topics of artificial intelligenc...'
   Top K: 3, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
⚠️ No relevant contexts found

✅ ANSWER
I could not find relevant information to answer this question.

📄 SOURCES (0 documents)
  No sources available

🤖 Model: N/A | Contexts: 0


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUERY 2/3
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

──────────────────────────────────────────────────────────────────────
📝 Query: Explain the key findings from the research papers about neural networks.
──────────────────────────────────────────────────────────────────────

🔍 Retrieving documents for query: 'Explain the key findings from the research papers ...'
   Top K: 3, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
⚠️ No relevant contexts found

✅ ANSWER
I could not find relevant information to answer this question.

📄 SOURCES (0 documents)
  No sources available

🤖 Model: N/A | Contexts: 0


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUERY 3/3
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

──────────────────────────────────────────────────────────────────────
📝 Query: What methodologies are discussed regarding agentic ai?
──────────────────────────────────────────────────────────────────────

🔍 Retrieving documents for query: 'What methodologies are discussed regarding agentic...'
   Top K: 3, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 1 documents (after filtering)

✅ ANSWER
The methodologies discussed regarding Agentic AI include Architectural Approaches, Learning Paradigms, Training Techniques, and Tools [1]. These methodologies are outlined in Figure 3 of the provided source, which gives an overview of Agentic AI Development Methodologies [1].

📄 SOURCES (1 documents)
  [1] Agentic_AI_Autonomous_Intelligence_for_Complex_GoalsA_Comprehensive_Survey.pdf (Page 8) - Relevance: 52.2%

🤖 Model: llama-3.3-70b-versatile | Contexts: 1


📊 SESSION STATISTICS
Total Queries: 10
Total Contexts Retrieved: 1
Avg Contexts per Query: 0.1

✅ RAG System ready for interactive queries!


In [5]:
# ============================================================================
# INTERACTIVE QUERY MODE
# ============================================================================

print("\n💬 Interactive Mode - Ask questions about your documents!")
print("   (Type 'exit' to stop)\n")

while True:
    user_query = input("Your question: ").strip()
    
    if user_query.lower() in ['exit', 'quit', 'q', '']:
        print("\n👋 Goodbye!")
        break
    
    response = rag_system.query(
        question=user_query,
        top_k=3,
        score_threshold=0.5,
        model="llama-3.3-70b-versatile",
        temperature=0.3,
        max_tokens=800
    )
    
    rag_system.display_response(response)

    # Add this cell after your interactive mode

def query_with_memory(rag_system, question: str, top_k: int = 3):
    """Query with awareness of previous conversation"""
    # Get last 2 exchanges
    history = rag_system.conversation_history[-2:] if rag_system.conversation_history else []
    
    history_context = ""
    if history:
        history_context = "\n\nPrevious conversation:\n"
        for conv in history:
            history_context += f"Q: {conv['question']}\nA: {conv['answer'][:100]}...\n"
    
    enhanced_question = f"{history_context}\n\nCurrent question: {question}"
    
    response = rag_system.query(
        question=enhanced_question,
        top_k=top_k,
        score_threshold=0.5
    )
    return response

# Test multi-turn conversation
print("Testing conversation memory...\n")
q1 = query_with_memory(rag_system, "What is RAG?")
rag_system.display_response(q1)

q2 = query_with_memory(rag_system, "What are its benefits?")
rag_system.display_response(q2)



💬 Interactive Mode - Ask questions about your documents!
   (Type 'exit' to stop)


──────────────────────────────────────────────────────────────────────
📝 Query: what is artificial intelligence?
──────────────────────────────────────────────────────────────────────

🔍 Retrieving documents for query: 'what is artificial intelligence?...'
   Top K: 3, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
⚠️ No relevant contexts found

✅ ANSWER
I could not find relevant information to answer this question.

📄 SOURCES (0 documents)
  No sources available

🤖 Model: N/A | Contexts: 0


──────────────────────────────────────────────────────────────────────
📝 Query: how to implement RAG system
──────────────────────────────────────────────────────────────────────

🔍 Retrieving documents for query: 'how to implement RAG system...'
   Top K: 3, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
⚠️ No relevant contexts found

✅ ANSWER
I could not find relevant information to answer this question.

📄 SOURCES (0 documents)
  No sources available

🤖 Model: N/A | Contexts: 0


──────────────────────────────────────────────────────────────────────
📝 Query: RAG
──────────────────────────────────────────────────────────────────────

🔍 Retrieving documents for query: 'RAG...'
   Top K: 3, Score threshold: 0.5
Generating embeddings for 1 texts...


...

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
⚠️ No relevant contexts found

✅ ANSWER
I could not find relevant information to answer this question.

📄 SOURCES (0 documents)
  No sources available

🤖 Model: N/A | Contexts: 0


👋 Goodbye!
Testing conversation memory...


──────────────────────────────────────────────────────────────────────
📝 Query: 

Previous conversation:
Q: hello
A: I could not find relevant information to answer this question....
Q: What is ML
A: I could not find relevant information to answer this question....


Current question: What is RAG?
──────────────────────────────────────────────────────────────────────

🔍 Retrieving documents for query: '

Previous conversation:
Q: hello
A: I could not f...'
   Top K: 3, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
⚠️ No relevant contexts found

✅ ANSWER
I could not find relevant information to answer this question.

📄 SOURCES (0 documents)
  No sources available

🤖 Model: N/A | Contexts: 0


──────────────────────────────────────────────────────────────────────
📝 Query: 

Previous conversation:
Q: What is ML
A: I could not find relevant information to answer this question....
Q: 

Previous conversation:
Q: hello
A: I could not find relevant information to answer this question....
Q: What is ML
A: I could not find relevant information to answer this question....


Current question: What is RAG?
A: I could not find relevant information to answer this question....


Current question: What are its benefits?
──────────────────────────────────────────────────────────────────────

🔍 Retrieving documents for query: '

Previous conversation:
Q: What is ML
A: I could ...'
   Top K: 3, Score threshold: 0.5
Generating embe

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
⚠️ No relevant contexts found

✅ ANSWER
I could not find relevant information to answer this question.

📄 SOURCES (0 documents)
  No sources available

🤖 Model: N/A | Contexts: 0



In [7]:
# %% Install (run once)
# !pip install pandas

# %% SAVE & LOAD SYSTEM - Recommended Approach

import json
import sqlite3
import os
from datetime import datetime

class RAGSystemSaver:
    """Simple, secure way to save/load your RAG system"""
    
    def __init__(self, save_dir="rag_saves"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
    
    def save_all(self, rag_system, name="default"):
        """Save everything in one call"""
        
        # 1. Save configuration as JSON
        config = {
            'name': name,
            'created_at': datetime.now().isoformat(),
            'vector_store': {
                'path': rag_system.vector_store.persist_directory,
                'collection_name': rag_system.vector_store.collection_name,
                'total_documents': rag_system.vector_store.collection.count()
            },
            'embedding_model': rag_system.embedding_manager.model_name,
            'total_conversations': len(rag_system.conversation_history)
        }
        
        with open(f"{self.save_dir}/{name}_config.json", 'w') as f:
            json.dump(config, f, indent=2)
        
        # 2. Save conversations to SQLite
        db_path = f"{self.save_dir}/{name}.db"
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Create table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS conversations (
                id INTEGER PRIMARY KEY,
                question TEXT,
                answer TEXT,
                num_contexts INTEGER,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Insert conversations
        for conv in rag_system.conversation_history:
            cursor.execute('''
                INSERT INTO conversations (question, answer, num_contexts)
                VALUES (?, ?, ?)
            ''', (conv['question'], conv['answer'], conv['num_contexts']))
        
        conn.commit()
        conn.close()
        
        print(f"✅ Saved to {self.save_dir}/")
        print(f"   Config: {name}_config.json")
        print(f"   Conversations: {name}.db")

# Usage:
saver = RAGSystemSaver()
saver.save_all(rag_system, name="my_rag_session")


✅ Saved to rag_saves/
   Config: my_rag_session_config.json
   Conversations: my_rag_session.db


In [8]:
def expand_query(rag_system, question: str):
    """Generate alternative phrasings to improve retrieval"""
    prompt = f"""Generate 2 alternative phrasings of this question:
"{question}"

Return ONLY the 2 questions, one per line. No numbering."""
    
    response = rag_system.llm_client.client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=200
    )
    
    alternatives = response.choices[0].message.content.strip().split('\n')
    return [question] + alternatives[:2]

def multi_query_retrieve(rag_system, question: str, top_k: int = 3):
    """Retrieve using multiple query variations"""
    queries = expand_query(rag_system, question)
    
    all_contexts = []
    seen_docs = set()
    
    for q in queries:
        contexts = rag_system.retriever.retrieve(q, top_k=2, score_threshold=0.5)
        for ctx in contexts:
            doc_id = f"{ctx['metadata'].get('source_file')}_{ctx['metadata'].get('page')}"
            if doc_id not in seen_docs:
                all_contexts.append(ctx)
                seen_docs.add(doc_id)
    
    return all_contexts[:top_k]

# Test multi-query retrieval
print("Testing query expansion...\n")
expanded_question = "Compare different RAG architectures"
better_contexts = multi_query_retrieve(rag_system, expanded_question)
print(f"Retrieved {len(better_contexts)} unique contexts using query expansion")


Testing query expansion...

🔍 Retrieving documents for query: 'Compare different RAG architectures...'
   Top K: 2, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
🔍 Retrieving documents for query: 'What are the key differences between various RAG a...'
   Top K: 2, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
🔍 Retrieving documents for query: 'How do distinct RAG architectures stack up against...'
   Top K: 2, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
Retrieved 0 unique contexts using query expansion


In [9]:
def query_with_confidence_check(rag_system, question: str, top_k: int = 3):
    """Query with confidence threshold warnings"""
    response = rag_system.query(question, top_k=top_k, score_threshold=0.5)
    
    if response.get('sources'):
        avg_similarity = np.mean([src['similarity'] for src in response['sources']])
        
        if avg_similarity < 0.6:
            print(f"⚠️ WARNING: Low confidence score ({avg_similarity:.1%})")
            print("   Results may be unreliable. Try a different question.\n")
        elif avg_similarity < 0.75:
            print(f"ℹ️ MODERATE confidence ({avg_similarity:.1%})")
        else:
            print(f"✅ HIGH confidence ({avg_similarity:.1%})\n")
    
    return response

# Test
response = query_with_confidence_check(rag_system, "What are the main findings?")
rag_system.display_response(response)



──────────────────────────────────────────────────────────────────────
📝 Query: What are the main findings?
──────────────────────────────────────────────────────────────────────

🔍 Retrieving documents for query: 'What are the main findings?...'
   Top K: 3, Score threshold: 0.5
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Generated embeddings with shape: (1, 384)
✅ Retrieved 0 documents (after filtering)
⚠️ No relevant contexts found

✅ ANSWER
I could not find relevant information to answer this question.

📄 SOURCES (0 documents)
  No sources available

🤖 Model: N/A | Contexts: 0

